In [6]:
import logging
import sys
import requests
from ct.aws import get_secret
from ct.utils import plt
from ct.utils import glassnode
from ct.utils import DF,npa,np,pd
import backtrader as bt
import ipdb
import time
from datetime import timedelta
from datetime import datetime as dt
from ccxtbt1 import CCXTStore
from ct.utils import qbars,date
from backtrader_plotting.schemes import Blackly
from backtrader_plotting.schemes import Tradimo
from backtrader_plotting.bokeh.live.plotlistener import PlotListener
from ct.alfafactory import trade_list,WalletObserver,SumWalletObserver,CloseObserver,PositionObserver,printTradeAnalysis
from ct.fs import addbb2

In [13]:
PRINTCANDLE=True
class GlassnodeStrategy(bt.Strategy):
    params = (('sym','BTC'),('betsize',0.001),('lags', 20),('delay',4),('trailpercent', 3),('inflowlimit',5),('inflowlimit2',12),('nstd',2))# ('entryz', 1.2), ('exitz',1.2),('methodma','ewm'),('methodstd','ewm'),('trailpercent', 2),('liqlimit',35))
    
    def notify_order(self, order): 
        if not order.alive():
            self.order = None  # No pending orders
        logging.critical(f"|notify_order|{order}|position={self.position.size}|brokerpos={self.broker.getposition(self.datas[0]).size}|c={self.data0.close[0]}|o={self.data0.open[0]}")
    def __init__(self):
        print("strat init()")
        self.order = None
        
    def start(self):
        print("strat start()")
        self.order = None  # Avoid operrations on pending order

    def next(self):
        if not BACKTEST:
            if self.live_data==False:
                return
        if self.order:
            return  # pending order execution

        curdt=bt.num2date(self.data0.datetime[0])
        o,h,l,c,v=self.data0.open[0],self.data0.high[0],self.data0.low[0],self.data0.close[0],self.data0.volume[0]
        if PRINTCANDLE:print(f"curdt={curdt} c={c}")
        
        if BACKTEST:
            dfpn=dfgsi#[::-1]
            dfp=dfpn[dfpn.index<curdt].head(self.p.lags)['v']
        else:
            dfp=glassnode('https://api.glassnode.com/v1/metrics/transactions/transfers_volume_to_exchanges_mean',a=self.p.sym,i='10m',s=dt.now()-timedelta(hours=2),Nretry=10)['v']
        if PRINTCANDLE:print(f"\n {dt.now()} next() self.dt={self.datetime.datetime(0)} data0close={self.data0.close[0]} data1close={self.data1.close[0] if hasattr(self,'data1') else 'not present'}  data1close={self.data2.close[0] if hasattr(self,'data2') else 'not present'}  \n")    
        if PRINTCANDLE:print(f"data0 get5={self.data0.close.get(size=5) if hasattr(self,'data0') else 'not present'}")

        if self.position.size==0: # not in the market
            if PRINTCANDLE:print(f"dfp.iloc[0] {dfp.iloc[0]}")
            if BACKTEST:
                condition=(len(dfp)>5 and dfp.iloc[self.p.delay]>self.p.inflowlimit and dfp.iloc[self.p.delay]<self.p.inflowlimit2)
            else:
                logging.error(f'\n{dfp}')
                condition=(dfp.iloc[-1]>self.p.inflowlimit and dfp.iloc[-1]<self.p.inflowlimit2)
            print('condition={}'.format(condition))        
            if condition :
                self.order = self.sell(data=self.data0,exectype=bt.Order.Market,size=self.p.betsize)
                
                logging.critical(f"{self.order}")
                self.order = None

        elif self.order is None: # Position in Market
            self.order = self.buy(data=self.data0,exectype=bt.Order.StopTrail,trailpercent=self.p.trailpercent/100,price=self.data0.open[0],size=self.p.betsize,params={'reduceOnly':True,'callbackRate':self.p.trailpercent}) #If no price is specified, the latest close price is used
            logging.critical(f"{self.order}")

    def notify_data(self, data, status, *args, **kwargs):
        if data._getstatusname(status) == 'LIVE':
            self.live_data = True
        else:
            self.live_data = False

    def log(self,txt,dummy=True):
        logging.debug(txt)

In [11]:
API_KEY = '1wvlrc0JsEMIWF6enI69cWiOojq'

def glassnode(url='https://api.glassnode.com/v1/metrics/transactions/transfers_volume_to_exchanges_mean',a='BTC',i='10m',s=1629500000):
    res = requests.get(url,params={'a': a,'i':i,s:s,'api_key': API_KEY}) #, 's':1629500000
    df = pd.read_json(res.text, convert_dates=['t']).set_index('t') 
    return df
                       
dfgsi=glassnode('https://api.glassnode.com/v1/metrics/transactions/transfers_volume_to_exchanges_mean',a='BTC',i='10m')[::-1]

In [8]:
display(dfgsi.head())

,v
t,
2021-10-13 09:40:00,0.1669669135
2021-10-13 09:30:00,0.5451063714
2021-10-13 09:20:00,0
2021-10-13 09:10:00,0.2889704277
2021-10-13 09:00:00,0.3040924188


In [14]:
BOTNAME='GLASSNODEMINIMAL'

LOGFILENAME=f'/core/logs/{BOTNAME}.log'#'/core/logs/livetestminimal.log'

logging.basicConfig(level=logging.ERROR,filename=LOGFILENAME,format="{asctime}| [{levelname:4}]|{message}|",style="{",force=True)

root = logging.getLogger()
#root.setLevel(logging.ERROR)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.ERROR)
root.addHandler(handler)
#handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))

MONITORPORT=50005

COIN_TARGET = "ETH"
COIN_REFER = "USDT"

DEBUG = True
BACKTEST=True

LAGS=3
MINUTES=5





def main():
    cerebro = bt.Cerebro(stdstats=False) #quicknotify=True)
    cerebro.addstrategy(GlassnodeStrategy,sym=COIN_TARGET,betsize=0.002,lags=20,delay=4,trailpercent=3,inflowlimit=5,inflowlimit2=12)
    cerebro.addobserver(bt.observers.Trades)
    cerebro.addobserver(bt.observers.BuySell,barplot=True,bardist=0.02)
    cerebro.addanalyzer(bt.analyzers.TimeReturn)#,_name='timereturn')
    
    if BACKTEST is False: 
        cerebro.addobserver(WalletObserver,ccy='USDT')
        cerebro.addobserver(PositionObserver,ccys=['USDT'])
        cerebro.addlistener(bt.listeners.RecorderListener)
        cerebro.addlistener(PlotListener, volume=False, scheme=Tradimo(hovertool_timeformat='%F %R:%S', tabs='single'), lookback=480,http_port=MONITORPORT,address='0.0.0.0',debug=True)
        srt=get_secret('b1-wzxNSw')
        broker_config = { 'apiKey': srt['k'],'secret': srt['s'], 'nonce': lambda: str(int(time.time() * 1000)), 'enableRateLimit': True,'options': {'defaultType': 'future'}, }
        store = CCXTStore(exchange='binance', currency=COIN_REFER, config=broker_config, retries=5, debug=False)
        broker_mapping ={ 'order_types': { bt.Order.Market: 'market', bt.Order.Limit: 'limit', bt.Order.Stop: 'stop-loss', bt.Order.StopLimit: 'STOP_LOSS_LIMIT',
                                          bt.Order.StopTrail:'TRAILING_STOP_MARKET'},#bt.Order.StopTrailLimit:''}, 
         'mappings': {'closed_order': {'key': 'status','value': 'closed'},'canceled_order': {'key': 'status','value': 'canceled' } } }
        hist_start_date = dt.utcnow() - timedelta(minutes=LAGS*MINUTES)
        data = store.getdata( dataname=f'{COIN_TARGET}/{COIN_REFER}' , name=f'{COIN_TARGET}/{COIN_REFER}', timeframe=bt.TimeFrame.Minutes,fromdate=hist_start_date, compression=MINUTES, ohlcv_limit=LAGS*MINUTES, drop_newest=True )
        cerebro.adddata(data)#,timeframe=bt.TimeFrame.Minutes,compression=MINUTES)
        broker = store.getbroker(broker_mapping=broker_mapping)
        print('pos:',broker.getposition(data).size)
        print('possize',broker.positions[COIN_TARGET+'USDT'].size)
        print('balance:',broker.get_balance())
        print('open orders:',broker.private_end_point(type='GET',endpoint='/openOrders',params={'symbol':"BTCUSDT"}))
        print(broker.private_end_point(type='GET',endpoint='/openOrders',params={'symbol':"BTCUSDT"}))
        cerebro.setbroker(broker)
    elif BACKTEST:
        cerebro.addobserver(bt.observers.Value)
        cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
        cerebro.addanalyzer(trade_list,_name='tradelist')
        cerebro.addanalyzer(bt.analyzers.Transactions)
        dfbtc=qbars(COIN_TARGET+'USDT@binance',freq=str(MINUTES)+"Min",table='ppttrades',date=[date(2021,7,29),date(2021,8,4)],addwhere=', vol>0.01')
        dfbtc1=dfbtc#.loc['2021-05']
        data=bt.feeds.PandasData(dataname=dfbtc1.rename(columns={'o':'open','h':'high','l':'low','c':'close','v':'volume'}),openinterest=None,timeframe=bt.TimeFrame.Minutes,compression=MINUTES)
        cerebro.adddata(data)
        
    logging.critical("alfabot cerebro.run()")
    backtest = cerebro.run(runonce=False) if not BACKTEST else cerebro.run(runonce=False,tradehistory=True)
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % final_value)
    if BACKTEST:
        rcsave=plt.rcParams['figure.figsize']
        plt.rcParams['figure.figsize']=[30, 16]
        cerebro.plot(style='line', rowsmajor=1, rowsminor=1,volume=True, iplot=False, barup='green', bardown='grey',fmt_x_ticks = '%Y-%b-%d %H:%M',fmt_x_data = '%Y-%b-%d %H:%M')
        plt.rcParams['figure.figsize']=rcsave
        
        btres = backtest[0]
        for k in ['total','pnl']:#backtest[0].analyzers.ta.get_analysis():
            print (k,dict(backtest[0].analyzers.ta.get_analysis()[k]))
        printTradeAnalysis(cerebro, backtest[0].analyzers)
        print("amount,price,sid,symbol,value")
        display(DF(btres.analyzers.tradelist.get_analysis()).set_index('datein'))
        
try:
    main()
except KeyboardInterrupt:
    print("finished.")

query=system "l /core/data/db/binance";
0!select  o:first px,h:max px,l:min px,c:last px ,v:sum vol by date, symbol, 0D00:05 xbar exchtm from ppttrades where  date within 2021.07.29 2021.08.04,  symbol=`ETHUSDT , vol>0.01
query=system "l /core/data/rdb/binance";
0!select  o:first px,h:max px,l:min px,c:last px ,v:sum vol by date, symbol, 0D00:05 xbar exchtm from ppttrades where  date within 2021.07.29 2021.08.04,  symbol=`ETHUSDT , vol>0.01
alfabot cerebro.run()
strat init()
strat start()
curdt=2021-07-28 23:55:00 c=2300.08

 2021-10-13 10:36:03.730151 next() self.dt=2021-07-28 23:55:00 data0close=2300.08 data1close=not present  data1close=not present  

data0 get5=array('d')
dfp.iloc[0] 0.11541106284263901
condition=False
curdt=2021-07-29 00:00:00 c=2287.09

 2021-10-13 10:36:03.735392 next() self.dt=2021-07-29 00:00:00 data0close=2287.09 data1close=not present  data1close=not present  

data0 get5=array('d')
dfp.iloc[0] 0.11541106284263901
condition=False
curdt=2021-07-29 00:05:00 c=


 2021-10-13 10:36:03.925932 next() self.dt=2021-07-29 03:15:00 data0close=2282.8 data1close=not present  data1close=not present  

data0 get5=array('d', [2283.37, 2286.84, 2284.86, 2284.23, 2282.8])
dfp.iloc[0] 0.170671374051204
condition=False
curdt=2021-07-29 03:20:00 c=2279.15

 2021-10-13 10:36:03.931055 next() self.dt=2021-07-29 03:20:00 data0close=2279.15 data1close=not present  data1close=not present  

data0 get5=array('d', [2286.84, 2284.86, 2284.23, 2282.8, 2279.15])
dfp.iloc[0] 0.170671374051204
condition=False
curdt=2021-07-29 03:25:00 c=2274.5

 2021-10-13 10:36:03.936096 next() self.dt=2021-07-29 03:25:00 data0close=2274.5 data1close=not present  data1close=not present  

data0 get5=array('d', [2284.86, 2284.23, 2282.8, 2279.15, 2274.5])
dfp.iloc[0] 0.0
condition=False
curdt=2021-07-29 03:30:00 c=2277.28

 2021-10-13 10:36:03.940812 next() self.dt=2021-07-29 03:30:00 data0close=2277.28 data1close=not present  data1close=not present  

data0 get5=array('d', [2284.23, 2282


 2021-10-13 10:36:04.128721 next() self.dt=2021-07-29 06:40:00 data0close=2297.39 data1close=not present  data1close=not present  

data0 get5=array('d', [2301.81, 2296.73, 2297.83, 2299.39, 2297.39])
dfp.iloc[0] 0.38522791396670003
condition=False
curdt=2021-07-29 06:45:00 c=2297.28

 2021-10-13 10:36:04.133818 next() self.dt=2021-07-29 06:45:00 data0close=2297.28 data1close=not present  data1close=not present  

data0 get5=array('d', [2296.73, 2297.83, 2299.39, 2297.39, 2297.28])
dfp.iloc[0] 0.6860068326614981
condition=False
curdt=2021-07-29 06:50:00 c=2295.99

 2021-10-13 10:36:04.138897 next() self.dt=2021-07-29 06:50:00 data0close=2295.99 data1close=not present  data1close=not present  

data0 get5=array('d', [2297.83, 2299.39, 2297.39, 2297.28, 2295.99])
dfp.iloc[0] 0.6860068326614981
condition=False
curdt=2021-07-29 06:55:00 c=2293.04

 2021-10-13 10:36:04.143615 next() self.dt=2021-07-29 06:55:00 data0close=2293.04 data1close=not present  data1close=not present  

data0 get5=


 2021-10-13 10:36:04.330349 next() self.dt=2021-07-29 10:05:00 data0close=2312.01 data1close=not present  data1close=not present  

data0 get5=array('d', [2324.8, 2317.59, 2317.53, 2313.75, 2312.01])
dfp.iloc[0] 0.0
condition=False
curdt=2021-07-29 10:10:00 c=2314.57

 2021-10-13 10:36:04.335463 next() self.dt=2021-07-29 10:10:00 data0close=2314.57 data1close=not present  data1close=not present  

data0 get5=array('d', [2317.59, 2317.53, 2313.75, 2312.01, 2314.57])
dfp.iloc[0] 0.0
condition=False
curdt=2021-07-29 10:15:00 c=2319.54

 2021-10-13 10:36:04.340230 next() self.dt=2021-07-29 10:15:00 data0close=2319.54 data1close=not present  data1close=not present  

data0 get5=array('d', [2317.53, 2313.75, 2312.01, 2314.57, 2319.54])
dfp.iloc[0] 0.41683343318255206
condition=False
curdt=2021-07-29 10:20:00 c=2318.51

 2021-10-13 10:36:04.345234 next() self.dt=2021-07-29 10:20:00 data0close=2318.51 data1close=not present  data1close=not present  

data0 get5=array('d', [2313.75, 2312.01, 2


 2021-10-13 10:36:04.533076 next() self.dt=2021-07-29 13:30:00 data0close=2314.35 data1close=not present  data1close=not present  

data0 get5=array('d', [2309.0, 2311.1, 2313.25, 2313.19, 2314.35])
dfp.iloc[0] 0.508465491522556
condition=False
curdt=2021-07-29 13:35:00 c=2307.43

 2021-10-13 10:36:04.538285 next() self.dt=2021-07-29 13:35:00 data0close=2307.43 data1close=not present  data1close=not present  

data0 get5=array('d', [2311.1, 2313.25, 2313.19, 2314.35, 2307.43])
dfp.iloc[0] 0.5432451393865031
condition=False
curdt=2021-07-29 13:40:00 c=2312.43

 2021-10-13 10:36:04.543260 next() self.dt=2021-07-29 13:40:00 data0close=2312.43 data1close=not present  data1close=not present  

data0 get5=array('d', [2313.25, 2313.19, 2314.35, 2307.43, 2312.43])
dfp.iloc[0] 0.5432451393865031
condition=False
curdt=2021-07-29 13:45:00 c=2312.66

 2021-10-13 10:36:04.548316 next() self.dt=2021-07-29 13:45:00 data0close=2312.66 data1close=not present  data1close=not present  

data0 get5=array

...


 2021-10-13 10:36:12.525275 next() self.dt=2021-08-04 14:40:00 data0close=2625.96 data1close=not present  data1close=not present  

data0 get5=array('d', [2617.18, 2624.3, 2614.51, 2617.87, 2625.96])
dfp.iloc[0] 0.45489308258620603
condition=False
curdt=2021-08-04 14:45:00 c=2636.23

 2021-10-13 10:36:12.530347 next() self.dt=2021-08-04 14:45:00 data0close=2636.23 data1close=not present  data1close=not present  

data0 get5=array('d', [2624.3, 2614.51, 2617.87, 2625.96, 2636.23])
dfp.iloc[0] 0.0
condition=False
curdt=2021-08-04 14:50:00 c=2635.34

 2021-10-13 10:36:12.535393 next() self.dt=2021-08-04 14:50:00 data0close=2635.34 data1close=not present  data1close=not present  

data0 get5=array('d', [2614.51, 2617.87, 2625.96, 2636.23, 2635.34])
dfp.iloc[0] 0.0
condition=False
curdt=2021-08-04 14:55:00 c=2637.38

 2021-10-13 10:36:12.540432 next() self.dt=2021-08-04 14:55:00 data0close=2637.38 data1close=not present  data1close=not present  

data0 get5=array('d', [2617.87, 2625.96, 26


 2021-10-13 10:36:12.728344 next() self.dt=2021-08-04 18:05:00 data0close=2691.58 data1close=not present  data1close=not present  

data0 get5=array('d', [2698.0, 2675.46, 2678.77, 2685.45, 2691.58])
dfp.iloc[0] 0.28339088730392104
condition=False
curdt=2021-08-04 18:10:00 c=2687.92

 2021-10-13 10:36:12.733367 next() self.dt=2021-08-04 18:10:00 data0close=2687.92 data1close=not present  data1close=not present  

data0 get5=array('d', [2675.46, 2678.77, 2685.45, 2691.58, 2687.92])
dfp.iloc[0] 0.28339088730392104
condition=False
curdt=2021-08-04 18:15:00 c=2684.91

 2021-10-13 10:36:12.738414 next() self.dt=2021-08-04 18:15:00 data0close=2684.91 data1close=not present  data1close=not present  

data0 get5=array('d', [2678.77, 2685.45, 2691.58, 2687.92, 2684.91])
dfp.iloc[0] 0.5534710971989171
condition=False
curdt=2021-08-04 18:20:00 c=2683.0

 2021-10-13 10:36:12.743456 next() self.dt=2021-08-04 18:20:00 data0close=2683.0 data1close=not present  data1close=not present  

data0 get5=ar


 2021-10-13 10:36:12.932440 next() self.dt=2021-08-04 21:30:00 data0close=2706.84 data1close=not present  data1close=not present  

data0 get5=array('d', [2717.48, 2706.71, 2708.65, 2713.78, 2706.84])
dfp.iloc[0] 0.0
condition=False
curdt=2021-08-04 21:35:00 c=2709.8

 2021-10-13 10:36:12.937492 next() self.dt=2021-08-04 21:35:00 data0close=2709.8 data1close=not present  data1close=not present  

data0 get5=array('d', [2706.71, 2708.65, 2713.78, 2706.84, 2709.8])
dfp.iloc[0] 0.17867739805335903
condition=False
curdt=2021-08-04 21:40:00 c=2715.65

 2021-10-13 10:36:12.942568 next() self.dt=2021-08-04 21:40:00 data0close=2715.65 data1close=not present  data1close=not present  

data0 get5=array('d', [2708.65, 2713.78, 2706.84, 2709.8, 2715.65])
dfp.iloc[0] 0.17867739805335903
condition=False
curdt=2021-08-04 21:45:00 c=2718.11

 2021-10-13 10:36:12.947680 next() self.dt=2021-08-04 21:45:00 data0close=2718.11 data1close=not present  data1close=not present  

data0 get5=array('d', [2713.7

total {'total': 1, 'open': 0, 'closed': 1}
pnl {'gross': AutoOrderedDict([('total', -0.027569399999999404), ('average', -0.027569399999999404)]), 'net': AutoOrderedDict([('total', -0.027569399999999404), ('average', -0.027569399999999404)])}
Backtesting Results
  Open Positions : -
  Closed Trades : 1
  Inital Portfolio Value : $10000
  Final Portfolio Value : $9999.972430599999
  Net P/L : $-0.03
  P/L Average per trade : $-0.03
amount,price,sid,symbol,value


,ref,ticker,dir,pricein,dateout,priceout,chng%,pnl,pnl%,size,value,cumpnl,nbars,pnl/bar,mfe,mae,mfe%,mae%
datein,,,,,,,,,,,,,,,,,,
2021-08-03 14:20:00,3,,short,2517.43,2021-08-04 01:35:00,2531.2147,0.55,-0.0275694,-0,-0.002,-5.03486,-0.0275694,135,-0,67.16,-22.26,2.67,-0.88
